# Random Forest for Iris
## 学習
***
- ハイパーパラメータサーチにOptuna
- 各ハイパーパラメータの解説は[データ科学便覧-Scikit-learnによるランダムフォレスト-](https://data-science.gr.jp/implementation/iml_sklearn_random_forest.html)

## モデル選択
***
- テストデータ全体のAccuracyスコアの最大値

## 精度評価
***
- 特徴量重要度
- sepal widthごと
- petal widthごと

## 追加実装
***
- モデル安定性の確認
- 各ハイパーパラメータごとの学習/テストの精度評価（max_depth等）
- 各精度にデータ数も表示（barプロットのsecondary_y）
- sepal width ✕ petal width等の2次元での精度評価とデータ数（ヒートマップ）

In [1]:
import warnings
import plotly
import optuna
import numpy as np
import scipy as sp
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
np.random.seed(0)
optuna.logging.disable_default_handler() # 学習時のログを非表示

## メソッド

In [2]:
def sturges_rule(n: int) -> int:
    ''''スタージェスの公式'''
    return int(round(1 + np.log2(n)))

## データセット（iris）

In [3]:
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
print(df_iris['species'].unique())
print(df_iris.info()) # 欠測値補間必要なし
display(df_iris.head())

[setosa, versicolor, virginica]
Categories (3, object): [setosa, versicolor, virginica]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal length (cm)    150 non-null float64
sepal width (cm)     150 non-null float64
petal length (cm)    150 non-null float64
petal width (cm)     150 non-null float64
species              150 non-null category
dtypes: category(1), float64(4)
memory usage: 5.0 KB
None


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## データ調査

In [4]:
df_iris.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [5]:
print('count_setosa: {0}'.format(len(df_iris.query('species == "setosa"'))))
print('count_versicolor: {0}'.format(len(df_iris.query('species == "versicolor"'))))
print('count_virginica: {0}'.format(len(df_iris.query('species == "virginica"'))))

count_setosa: 50
count_versicolor: 50
count_virginica: 50


In [29]:
sturges_bins = sturges_rule(len(df_iris))

tr_sepal_length = go.Histogram(
    name='sepal length (cm)', 
    x=df_iris['sepal length (cm)'], 
    nbinsx=sturges_bins,
    opacity=0.75
)
tr_sepal_width = go.Histogram(
    name='sepal width (cm)', 
    x=df_iris['sepal width (cm)'], 
    nbinsx=sturges_bins,
    opacity=0.75
)
tr_petal_length = go.Histogram(
    name='petal length (cm)', 
    x=df_iris['petal length (cm)'], 
    nbinsx=sturges_bins,
    opacity=0.75
)
tr_petal_width = go.Histogram(
    name='petal width (cm)', 
    x=df_iris['petal width (cm)'], 
    nbinsx=sturges_bins,
    opacity=0.75
)

fig = tools.make_subplots(rows=2, cols=2)
fig.append_trace(tr_sepal_length, 1, 1)
fig.append_trace(tr_sepal_width, 1, 2)
fig.append_trace(tr_petal_length, 2, 1)
fig.append_trace(tr_petal_width, 2, 2)

fig['layout']['xaxis1'].update(title='cm')
fig['layout']['xaxis2'].update(title='cm')
fig['layout']['xaxis3'].update(title='cm')
fig['layout']['xaxis4'].update(title='cm')

fig['layout']['yaxis1'].update(title='度数')
fig['layout']['yaxis2'].update(title='度数')
fig['layout']['yaxis3'].update(title='度数')
fig['layout']['yaxis4'].update(title='度数')

fig['layout'].update(title='説明変数のヒストグラム', width=1600, height=1400,
                     font={"family":"Yu Gothic Bold, sans-selif", "size":25})
py.iplot(fig, show_link=False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



## 学習/テストデータ作成

In [7]:
df_iris_add_factrize = df_iris.copy(deep=False)
df_iris_add_factrize['species_factrize'] = pd.factorize(df_iris_add_factrize['species'])[0]
df_iris_add_factrize.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,species_factrize
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0


In [8]:
df_iris_add_is_train = df_iris_add_factrize.copy(deep=False)
df_iris_add_is_train['is_train'] = np.random.uniform(0, 1, len(df_iris_add_is_train)) <= 0.75
train, test = df_iris_add_is_train.query('is_train == True'), df_iris_add_is_train.query('is_train == False')
print(f'count train: {len(train)}')
print(f'count train: {len(test)}')
display(train.head())
display(test.head())

count train: 118
count train: 32


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,species_factrize,is_train
0,5.1,3.5,1.4,0.2,setosa,0,True
1,4.9,3.0,1.4,0.2,setosa,0,True
2,4.7,3.2,1.3,0.2,setosa,0,True
3,4.6,3.1,1.5,0.2,setosa,0,True
4,5.0,3.6,1.4,0.2,setosa,0,True


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,species_factrize,is_train
7,5.0,3.4,1.5,0.2,setosa,0,False
8,4.4,2.9,1.4,0.2,setosa,0,False
10,5.4,3.7,1.5,0.2,setosa,0,False
13,4.3,3.0,1.1,0.1,setosa,0,False
17,5.1,3.5,1.4,0.3,setosa,0,False


## 学習と予測

In [14]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1, 20),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 8, 16),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 60),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced']),
        'random_state': trial.suggest_int('random_state', 0, 0)
    }
    
    model = RandomForestClassifier(**params)
    model.fit(train.iloc[:,:4].values, train['species_factrize'].values)
    
    score = accuracy_score(test['species_factrize'], model.predict(test.iloc[:,:4].values))
    return 1.0 - score

study = optuna.create_study()
study.optimize(objective, n_trials=100)
print(f'best params: {study.best_params}')

best params: {'n_estimators': 2, 'criterion': 'gini', 'max_depth': 7, 'min_samples_split': 12, 'max_leaf_nodes': 44, 'class_weight': 'balanced', 'random_state': 0}


In [15]:
model = RandomForestClassifier(n_estimators=study.best_params['n_estimators'],
                               criterion=study.best_params['criterion'],
                               max_depth=study.best_params['max_depth'],
                               min_samples_split=study.best_params['min_samples_split'], 
                               class_weight=study.best_params['class_weight'],
                               random_state=study.best_params['random_state'])
model.fit(train.iloc[:,:4].values, train['species_factrize'].values)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=7, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=12, min_weight_fraction_leaf=0.0,
            n_estimators=2, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [16]:
pred_proba = model.predict_proba(test.iloc[:,:4].values) # 予測確率
test['proba_setosa'] = pred_proba.T[0]
test['proba_versicolor'] = pred_proba.T[1]
test['proba_virginica'] = pred_proba.T[2]
test['species_pred'] = model.predict(test.iloc[:,:4].values) # 予測ラベル
test.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,species_factrize,is_train,proba_setosa,proba_versicolor,proba_virginica,species_pred
7,5.0,3.4,1.5,0.2,setosa,0,False,1.0,0.0,0.0,0
8,4.4,2.9,1.4,0.2,setosa,0,False,1.0,0.0,0.0,0
10,5.4,3.7,1.5,0.2,setosa,0,False,1.0,0.0,0.0,0
13,4.3,3.0,1.1,0.1,setosa,0,False,1.0,0.0,0.0,0
17,5.1,3.5,1.4,0.3,setosa,0,False,1.0,0.0,0.0,0


## 特徴量重要度

In [17]:
fi = model.feature_importances_
df_fi = pd.DataFrame(fi, index=df_iris.iloc[:,:4].columns, columns=['feature_importance'])
df_fi.sort_values('feature_importance', ascending=False)

,feature_importance
petal width (cm),0.706706
petal length (cm),0.289978
sepal length (cm),0.003317
sepal width (cm),0.000000


## 精度評価

In [18]:
ar_all = accuracy_score(test['species_factrize'], test['species_pred'])
print(f'全体正解率: {ar_all}')

全体正解率: 0.96875


In [19]:
ar_setosa = accuracy_score(test.query('species_factrize == 0')['species_factrize'], test.query('species_factrize == 0')['species_pred'])
print(f'setosa正解率: {ar_setosa}')

setosa正解率: 1.0


In [20]:
ar_versicolor = accuracy_score(test.query('species_factrize == 1')['species_factrize'], test.query('species_factrize == 1')['species_pred'])
print(f'versicolor正解率: {ar_versicolor}')

versicolor正解率: 0.8571428571428571


In [21]:
ar_virginica = accuracy_score(test.query('species_factrize == 2')['species_factrize'], test.query('species_factrize == 2')['species_pred'])
print(f'virginica正解率: {ar_virginica}')

virginica正解率: 1.0


In [30]:
data = [go.Bar(
    x=['全体','setosa','versicolor','virginica'],
    y=[ar_all,ar_setosa,ar_versicolor,ar_virginica],
    opacity=0.75
)]
layout = go.Layout(
    title='全体/ラベルごとの正解率', 
    width=800, height=600,
    font={"family":"Yu Gothic Bold, sans-selif", "size":25},
    yaxis=dict(title='正解率')
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, show_link=False)

In [23]:
test['cut_sepal_width'] = pd.qcut(test['sepal width (cm)'], 4)
ar_sepal_width = test.groupby('cut_sepal_width').apply(lambda x: len(x.query('species_factrize == species_pred')) / len(x))
ar_sepal_width

cut_sepal_width
(2.1990000000000003, 2.875]    1.000000
(2.875, 3.05]                  1.000000
(3.05, 3.4]                    0.888889
(3.4, 3.8]                     1.000000
dtype: float64

In [31]:
data = [go.Bar(
    x=list(map(str, ar_sepal_width.index)),
    y=ar_sepal_width.values,
    opacity=0.75
)]
layout = go.Layout(
    title='sepal widthごとの正解率', 
    width=1800, height=600,
    font={"family":"Yu Gothic Bold, sans-selif", "size":25},
    xaxis=dict(title='cm'),
    yaxis=dict(title='正解率')
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, show_link=False)

In [25]:
test['cut_petal_width'] = pd.qcut(test['petal width (cm)'], 4)
ar_petal_width = test.groupby('cut_petal_width').apply(lambda x: len(x.query('species_factrize == species_pred')) / len(x))
ar_petal_width

cut_petal_width
(0.099, 0.3]    1.0
(0.3, 1.5]      1.0
(1.5, 1.925]    0.8
(1.925, 2.5]    1.0
dtype: float64

In [27]:
data = [go.Bar(
    x=list(map(str, ar_petal_width.index)),
    y=ar_petal_width.values,
    opacity=0.75
)]
layout = go.Layout(
    title='petal widthごとの正解率', 
    width=1800, height=600,
    font={"family":"Yu Gothic Bold, sans-selif", "size":25},
    xaxis=dict(title='cm'),
    yaxis=dict(title='正解率')
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, show_link=False)